In [14]:
import os
import sys
sys.path.append('../part12')
import numpy as np

from cv23_lab1_part3_utils import featuresSURF
from cv23_lab1_part3_utils import featuresHOG
from cv23_lab1_part3_utils import matching_evaluation
from cv23_lab1_part3_utils import FeatureExtraction
from cv23_lab1_part3_utils import createTrainTest
from cv23_lab1_part3_utils import BagOfWords
from cv23_lab1_part3_utils import svm
from corner_detection_utils import CornerDetection
from corner_detection_utils import HarrisLaplacian
from blob_detection_utils import BlobDetection
from blob_detection_utils import HessianLaplacian
from box_detection_utils import BoxLaplacian

# Image Matching

In [15]:
rotations = [-20, -10, 0, 10, 20]
escalations = [0.6, 0.8, 1.0, 1.2]
transformations = [(r, e) for r in rotations for e in escalations]

In [16]:
descriptors = [lambda I, kp: featuresSURF(I, kp), lambda I, kp: featuresHOG(I, kp)]
detectors = [lambda i: CornerDetection(i, 2, 2.5, 0.005, 0.05),
             lambda i: HarrisLaplacian(i, 2, 2.5, 0.005, 0.05, 1.1, 6),
             lambda i: BlobDetection(i, 2, 0.005),
             lambda i: HessianLaplacian(i, 2, 0.005, 1.1, 6),
             lambda i: BoxLaplacian(i, 2, 0.005, 1.1, 6)]
descriptor_names = ["featuresSURF", "featuresHOG"]
detector_names = ["CornerDetection", "HarrisLaplacian", "BlobDetection", "HessianLaplacian", "BoxLaplacian"]
detector_descriptor = [(detector, descriptor)
                       for detector in detectors
                       for descriptor in descriptors]
detector_descriptor_names = [(detector, descriptor)
                             for detector in detector_names
                             for descriptor in descriptor_names]

In [17]:
# this takes around 1-2 minutes
avg_scale_errors_list = np.zeros((len(descriptors), len(detectors), 3))
avg_theta_errors_list = np.zeros((len(descriptors), len(detectors), 3))
for index, descriptor in enumerate(descriptors):
    for jndex, detector in enumerate(detectors):
        avg_scale_errors_list[index, jndex], avg_theta_errors_list[index, jndex] = matching_evaluation(detector, descriptor)

In [18]:
with open("../report/avg_scale_errors.txt", 'w') as file:
    for index, descriptor in enumerate(descriptor_names):
        for jndex, detector in enumerate(detector_names):
            file.write(f"{descriptor}, {detector} :\t{avg_scale_errors_list[index, jndex]}\n")
with open("../report/avg_theta_errors.txt", 'w') as file:
    for index, descriptor in enumerate(descriptor_names):
        for jndex, detector in enumerate(detector_names):
            file.write(f"{descriptor}, {detector} :\t{avg_theta_errors_list[index, jndex]}\n")

# Feature Extraction

In [19]:
files = ["./features/"+detector+'_'+descriptor+".txt"
               for detector in detector_names
               for descriptor in descriptor_names]
#for name in files:
#    os.mknod(name)
functions_and_files = list(zip(detector_descriptor, files))

In [20]:
# this takes around 20 minutes
# DO NOT EXECUTE THIS CELL IF THE "FEATURES" DIRECTORY EXISTS
# IF IT EXISTS, EXECUTE THE NEXT CELL.
features_list = [FeatureExtraction(detector, descriptor, saveFile=store)
                 for ((detector, descriptor), store) in functions_and_files]

Time for feature extraction: 32.985
Time for feature extraction: 25.580
Time for feature extraction: 210.221
Time for feature extraction: 150.143
Time for feature extraction: 21.750
Time for feature extraction: 31.316
Time for feature extraction: 169.615
Time for feature extraction: 124.148
Time for feature extraction: 175.882
Time for feature extraction: 117.983


In [21]:
# this is instant provided that the files exist
features_list = [FeatureExtraction(detector, descriptor, loadFile=store) 
                   for ((detector, descriptor), store) in functions_and_files]

In [66]:
for index, feats in enumerate(features_list):
    accs = []
    for k in range(5):
        # Split into a training set and a test set.
        data_train, label_train, data_test, label_test = createTrainTest(feats, k)

        # Perform Kmeans to find centroids for clusters.
        BOF_tr, BOF_ts = BagOfWords(data_train, data_test)

        # Train an svm on the training set and make predictions on the test set
        acc, preds, probas = svm(BOF_tr, label_train, BOF_ts, label_test)
        accs.append(acc)
    detector, descriptor = detector_descriptor_names[index]
    with open(f"mean_acc_{detector}_{descriptor}.txt", 'w') as file:
        file.write(f"{100.0*np.mean(acc):.3f}")